## Use CrewAI

- [GitHub](https://github.com/joaomdmoura/crewai/)
- [Docs](https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/)

### Quick Overview

Top-down approach
- Goal: What is the crew for? (define the use-case)
- Define all the Tasks required to achieve the goal? (including human review/action)
- Define and Assign the agent(s) required to complete each task
- Define Tools (maybe used by Agent/Task)
- Kickoff Crew process

In [1]:
from api_key_store import ApiKeyStore
s = ApiKeyStore()

openai_api_key = s.get_api_key(provider="OPENAI")
serp_api_key = s.get_api_key(provider="SerpApi")

In [2]:
s.api_providers

dict_keys(['OPENAI', 'GOOGLE', 'HUGGING_FACE', 'WOLFRAM_ALPHA', 'ANTHROPIC', 'COHERE', 'LlamaIndex', 'SerpApi', 'Browserless', 'DeepL', 'PANDASAI', 'GITHUB'])

In [3]:
import os
os.environ["SERPER_API_KEY"] = serp_api_key
os.environ["OPENAI_API_KEY"] = openai_api_key

In [4]:
from crewai import Agent
from crewai_tools import SerperDevTool

In [5]:
import crewai

### Enter topic of interest

In [6]:
topic = 'AI Research by Google'

### Step 1: Assemble Your Agents

#### Define Tools

In [7]:
search_tool = SerperDevTool()

#### Define Agents

In [8]:
# Creating a senior researcher agent with memory and verbose mode
researcher = Agent(
  role='Senior Researcher',
  goal='Uncover groundbreaking technologies in {topic}',
  verbose=True,
  memory=True,
  backstory=(
    "Driven by curiosity, you're at the forefront of"
    "innovation, eager to explore and share knowledge that could change"
    "the world."
  ),
  tools=[search_tool],
  allow_delegation=True
)

In [9]:
# Creating a writer agent with custom tools and delegation capability
writer = Agent(
  role='Writer',
  goal='Narrate compelling tech stories about {topic}',
  verbose=True,
  memory=True,
  backstory=(
    "With a flair for simplifying complex topics, you craft"
    "engaging narratives that captivate and educate, bringing new"
    "discoveries to light in an accessible manner."
  ),
  tools=[search_tool],
  allow_delegation=False
)

### Step 2: Define the Tasks

In [10]:
from crewai import Task

# Research task
research_task = Task(
  description=(
    "Identify the next big trend in {topic}."
    "Focus on identifying pros and cons and the overall narrative."
    "Your final report should clearly articulate the key points,"
    "its market opportunities, and potential risks."
  ),
  expected_output='A comprehensive 3 paragraphs long report on the latest AI trends.',
  tools=[search_tool],
  agent=researcher,
)

# Writing task with language model configuration
write_task = Task(
  description=(
    "Compose an insightful article on {topic}."
    "Focus on the latest trends and how it's impacting the industry."
    "This article should be easy to understand, engaging, and positive."
  ),
  expected_output='A 4 paragraph article on {topic} advancements formatted as markdown.',
  tools=[search_tool],
  agent=writer,
  async_execution=False,
  output_file='new-blog-post.md'  # Example of output customization
)

### Step 3: Form the Crew

In [11]:
from crewai import Crew, Process

# Forming the tech-focused crew with some enhanced configurations
crew = Crew(
  agents=[researcher, writer],
  tasks=[research_task, write_task],
  process=Process.sequential,  # Optional: Sequential task execution is default
  memory=False,  # True,
  cache=False,   # True,
  max_rpm=100,
  # share_crew=True
)


Issue:
```
OperationalError                          Traceback (most recent call last)
File ~/.local/lib/python3.10/site-packages/sqlalchemy/engine/base.py:1967, in Connection._exec_single_context(self, dialect, context, statement, parameters)
   1966     if not evt_handled:
-> 1967         self.dialect.do_execute(
   1968             cursor, str_statement, effective_parameters, context
   1969         )
   1971 if self._has_events or self.engine._has_events:
```
root-cause:

```
which jupyter
/home/gongai/.local/bin/jupyter

```

### Step 4: Kick It Off

In [12]:
# Starting the task execution process with enhanced feedback
result = crew.kickoff(inputs={'topic': topic})
print(result)



> Entering new CrewAgentExecutor chain...
To begin this task, I need to get the latest information on AI Research by Google. The best tool to use for this would be the "Search the internet" tool to find recent articles, blog posts or press releases about Google's AI research projects.

Action: Search the internet
Action Input: {"search_query": "latest AI Research by Google"} 


Search results: Title: Google Research - Explore Our Latest Research in Science and AI
Link: http://research.google/
Snippet: Discover Google Research. We publish research papers across a wide range of domains and share our latest developments in AI and science research.
---
Title: Research - Google AI
Link: https://ai.google/discover/research/
Snippet: RESEARCH: Tackling the most challenging problems in computer science. Our teams aspire to make discoveries that positively impact society.
---
Title: Making AI helpful for everyone - Google AI – Google AI
Link: https://ai.google/
Snippet: Our AI innovations are

RateLimitError: Error code: 429 - {'error': {'message': 'Your account is not active, please check your billing details on our website.', 'type': 'billing_not_active', 'param': None, 'code': 'billing_not_active'}}